<a href="https://colab.research.google.com/github/georgedouzas/avatar-poc/blob/main/environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup & Installation

In [ ]:
# Setup environment
!sudo apt-get update
!apt install software-properties-common
!sudo apt-get install python3.8 python3.8-distutils
!curl -sS https://bootstrap.pypa.io/pip/3.8/get-pip.py -o get-pip.py
!sudo python3.8 get-pip.py
!python3.8 -m pip install -U setuptools wheel
!sudo apt-get install python3.9 python3.9-distutils
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!sudo python3.9 get-pip.py
!python3.9 -m pip install -U setuptools wheel

# Install SadTalker
!git clone https://github.com/cedro3/SadTalker.git &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt
!rm -rf checkpoints
!bash scripts/download_models.sh

# Install piper
!python3.9 -m pip install piper-tts
!python3.9 -m piper.download_voices en_US-lessac-medium
!python3.9 -m piper.download_voices el_GR-rapunzelina-low

# Gradio App

In [ ]:
languages_mapping = {
    'English': 'en_US-lessac-medium',
    'Greek': 'el_GR-rapunzelina-low'
}
language = languages_mapping['English']
text = 'My name is Quento. I am a great company'
driven_audio = 'examples/driven_audio/' + 'test.wav'
source_image = 'examples/source_image/' + 'full3.png'

!python3.9 -m piper -m $language -f examples/driven_audio/test.wav -- $text
!python3.8 inference.py --driven_audio $driven_audio --source_image $source_image --result_dir ./results

In [ ]:
import subprocess
import wave
import gradio as gr

def text_to_speech(text):
    output_path = 'examples/driven_audio/output.wav'
    subprocess.run(['python3.9', '-m', 'piper', '-m', 'en_US-lessac-medium', '-f', output_path, '--', text], stdout=subprocess.PIPE)
    return output_path

with gr.Blocks() as app:
    gr.Markdown("# Text to Speech")
    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(lines=5, placeholder="Enter text here...", label="Text")
            submit_btn = gr.Button("Generate Speech", variant="primary")
        with gr.Column():
            audio_output = gr.Audio(label="Generated Audio", type="filepath")

    submit_btn.click(text_to_speech, inputs=text_input, outputs=audio_output)